<a href="https://colab.research.google.com/github/Tennedar/yt-clipper-pro/blob/main/YouTube_1080p_Clipper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. 安装下载工具和网页框架
!pip install -q yt-dlp streamlit
# 2. 安装内网穿透工具（让你可以通过公网链接打开 Colab 内部的网页）
!npm install -g localtunnel

In [ ]:
import urllib
import os

# --- 1. 自动写入网页逻辑 ---
with open('app.py', 'w') as f:
    f.write("""
import streamlit as st
import os, subprocess, json

st.set_page_config(page_title="Colab Pro Clipper", page_icon="🎬", layout="wide")
st.title("🎬 YouTube 1080p 极速剪辑器")
st.markdown("利用 Google Cloud 算力实现无损剪辑，不消耗本地流量。")

url = st.text_input("第一步：输入 YouTube 链接", placeholder="https://www.youtube.com/watch?v=...")

if url:
    with st.spinner("🔍 正在解析视频元数据..."):
        # 获取视频时长和标题
        res = subprocess.run(['yt-dlp', '--dump-json', '--no-playlist', url], capture_output=True, text=True)
        if res.returncode == 0:
            info = json.loads(res.stdout)
            duration = info.get('duration', 0)
            title = info.get('title', 'video')
            st.success(f"✅ 已找到视频: {title}")

            # 自动计算时间默认值
            h = int(duration // 3600)
            m = int((duration % 3600) // 60)
            s = int(duration % 60)
            default_end = f"{h:02d}:{m:02d}:{s:02d}"

            st.write("---")
            st.write("第二步：设置剪辑时间点")
            c1, c2 = st.columns(2)
            start_t = c1.text_input("开始时间", value="00:00:00")
            end_t = c2.text_input("结束时间", value=default_end)

            if st.button("🚀 开始极速剪辑并下载"):
                output = "clip_1080p.mp4"
                if os.path.exists(output): os.remove(output)

                # 执行下载和剪辑
                cmd = [
                    'yt-dlp', '-f', 'bestvideo[height<=1080]+bestaudio/best',
                    '--merge-output-format', 'mp4',
                    '--external-downloader', 'ffmpeg',
                    '--external-downloader-args', f'ffmpeg_i:-ss {start_t} -to {end_t}',
                    url, '-o', output
                ]
                with st.spinner("🚀 Google 服务器正在拼命合成中..."):
                    process = subprocess.run(cmd, capture_output=True, text=True)
                    if os.path.exists(output):
                        with open(output, "rb") as file:
                            st.download_button(
                                label="💾 点击保存 1080p 视频到本地",
                                data=file,
                                file_name=f"{title}_clip.mp4",
                                mime="video/mp4"
                            )
                    else:
                        st.error("下载失败，请检查视频是否受到限制。")
                        st.code(process.stderr)
        else:
            st.error("解析失败。可能原因：链接错误或视频需登录查看。")
""")

# --- 2. 打印访问凭证 ---
# 获取 Colab 的公网 IP，作为 localtunnel 的访问密码
ip = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip()
print(f"\n🔑 你的访问密码 (Endpoint IP) 是: {ip}")
print("👇 请点击下方链接进入网页，并在 Tunnel Password 框中填入上面的密码：\n")

# --- 3. 启动隧道 ---
!streamlit run app.py & npx localtunnel --port 8501